In [ ]:
# Install the needed packages
%pip install pathlib pandas numpy tensorflow tensorboard

In [ ]:
# Import packages.
import os
import signal
import tempfile
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorboard as tb

In [ ]:
# Define parameters.
LOGS_PATH = '../logs'
DATA_PATH = '../data/own_data_preparation.csv'
RESULT_COLUMN = 'result_team1'
RESULT_HEADERS = ['loss_team1', 'draw', 'win_team1']
RESULT_INFO_COLUMNS = ['season', 'date', 'team1', 'team2']
DUMMY_COLUMNS = ['team1', 'team2']
NORMAILZE_COLUMNS = ['points', 'squard',
    'average_age', 'average_market_value_in_euro', 'total_market_value_in_euro',
    'rank_last_season', 'points_last_season_all', 'points_last_season']
WITH_TEAMS = False
VALIDATION_SIZE = 0.2
LOSS_FUNCTION = 'sparse_categorical_crossentropy'
OPTIMIZER_FUNCTION = 'Adam'
METRICS = 'sparse_categorical_accuracy'
EPOCHS = 500
BATCH_SIZE = 32
TITLE = 'prediction'
LEARNING_RATE = 0.001

TENSORBOARD_PROCESS = 'tensorboard'
TENSORBOARD_SERVER = 'localhost'
TENSORBOARD_PORT = 6008

LOGS_PATH = os.path.join(LOGS_PATH, TITLE)

In [ ]:
# Set up tensorboard.
%load_ext tensorboard
logs = Path(LOGS_PATH)
logs.mkdir(mode=0o777, parents=True, exist_ok=True)

In [ ]:
# Load the data.
data = pd.read_csv(DATA_PATH, delimiter=',', decimal='.')

# Create the difference between team1 and team2 for each feature.
for column in NORMAILZE_COLUMNS:
    data[column] = data[column + '_team1'] - data[column + '_team2']

# Normalize column using the max value for each column.
for column in NORMAILZE_COLUMNS:
    max_value = data[column].max()
    data[column] = data[column].apply(lambda x: x / max_value)

# Get columns for result table.
result_info = data[data[RESULT_COLUMN].isna()]
result_info = result_info.loc[:, RESULT_INFO_COLUMNS]

# Get the wanted columns and create dummy columns for the teams.
if WITH_TEAMS:
    data_cal = data.loc[:, DUMMY_COLUMNS + NORMAILZE_COLUMNS + [RESULT_COLUMN]]
    data_cal = pd.get_dummies(data_cal, columns=DUMMY_COLUMNS)
else:
    data_cal = data.loc[:, NORMAILZE_COLUMNS + [RESULT_COLUMN]]

# Separate columns with match results from the ones that have none.
data_prediction = data_cal[data_cal[RESULT_COLUMN].isna()]
data_modeling = data_cal.dropna()

# Split the data set.
split_index = int(len(data_modeling) * VALIDATION_SIZE)
data_modeling = data_modeling.sample(frac=1)
data_train = data_modeling[split_index:]
data_valid = data_modeling[:split_index]

par_train = data_train.loc[:, data_train.columns != RESULT_COLUMN]
res_train = data_train.loc[:, [RESULT_COLUMN]]
par_valid = data_valid.loc[:, data_train.columns != RESULT_COLUMN]
res_valid = data_valid.loc[:, [RESULT_COLUMN]]
data_prediction = data_prediction.loc[:, data_train.columns != RESULT_COLUMN]

if WITH_TEAMS:
    dummies = [key for key in par_train.keys() if 'team' in key]

data_modeling.head()

In [ ]:
data_prediction.head()

In [ ]:
# Convert data frames to numpy arrays.
res_train = res_train.to_numpy()
res_valid = res_valid.to_numpy()

print(par_train.shape)
print(res_train.shape)
print(par_valid.shape)
print(res_valid.shape)

In [ ]:
def defineModel(input_dim):
    # Define the model
    nodes = input_dim*2-1
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(nodes, activation='relu', input_dim=input_dim),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    # Define the optimizer function.
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    # Compile the model.
    model.compile(loss=LOSS_FUNCTION, optimizer=optimizer, metrics=[METRICS])

    return model


In [ ]:
def runModel(model, x_train, x_valid, y_train, y_valid, title):
    # Define callback function for writing data for tensorBoard
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(LOGS_PATH, title), histogram_freq=1)

    # Run the model.
    history = model.fit(
        x=x_train,
        y=y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(x_valid, y_valid),
        callbacks=[tensorboard_callback],
        verbose=0
    )

    return history

In [ ]:
# Run the model.
sel_train = par_train.to_numpy()
sel_valid = par_valid.to_numpy()

model = defineModel(sel_train.shape[1])
history = runModel(model, sel_train, sel_valid, res_train, res_valid, TITLE)

In [ ]:
# TODO: Check if the order of the prediction data set is not changed by tensorflow => Otherwise the code below would merge the wrong infos to the predictions.
# TODO: Check if the order of predictions results is correct (loss, draw, win) => Otherwise the table headers, need to be changed.

# Make predictions.
result_info = result_info.to_numpy()
pre_data = data_prediction.to_numpy()

predictions = model.predict(pre_data)

# Show the predictions as table.
result = []
for i in range(0, len(predictions)):
    item = list(result_info[i]) + list(predictions[i])
    result.append(item)

result = pd.DataFrame(result, columns = RESULT_INFO_COLUMNS + RESULT_HEADERS)
result

In [ ]:
# Kill the existing tensorboard process and delete the tensorflow temp folder. After this start a new tensorboard process.
try:    
    # Iterating through each instance of the process.
    for line in os.popen("ps ax | grep " + TENSORBOARD_PROCESS + " | grep -v grep"):
        fields = line.split()
            
        # Extracting Process ID from the output.
        pid = fields[0]
            
        # Terminating process.
        os.kill(int(pid), signal.SIGKILL)

    # Delete tensorboard temp folder.
    tb_temp_folder = os.path.join(tempfile.gettempdir(), '.tensorboard-info')
    os.system("rm -rf "+tb_temp_folder)
    print("Process Successfully terminated") 
except Exception as e:
    print(e)

%tensorboard --logdir $LOGS_PATH --host $TENSORBOARD_SERVER --port $TENSORBOARD_PORT